In [17]:
!pip3 install datasets

In [3]:
from datasets import load_dataset

In [4]:
emotions = load_dataset("emotion")
print('----------------emotion dataset----------------')
print(emotions)   # check the emotion datasets

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/9.05k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/1.03M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/127k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/129k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/16000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

----------------emotion dataset----------------
DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
})


In [5]:
train_ds = emotions["train"]
emotions.set_format(type="pandas")
df_train = emotions["train"][:]
df_train

,text,label
0,i didnt feel humiliated,0
1,i can go from feeling so hopeless to so damned...,0
2,im grabbing a minute to post i feel greedy wrong,3
3,i am ever feeling nostalgic about the fireplac...,2
4,i am feeling grouchy,3
...,...,...
15995,i just had a very brief time in the beanbag an...,0
15996,i am now turning and i feel pathetic that i am...,0
15997,i feel strong and good overall,1
15998,i feel like this was such a rude comment and i...,3


In [6]:
test_ds = emotions["test"]
df_test = emotions["train"][:]
df_test

,text,label
0,i didnt feel humiliated,0
1,i can go from feeling so hopeless to so damned...,0
2,im grabbing a minute to post i feel greedy wrong,3
3,i am ever feeling nostalgic about the fireplac...,2
4,i am feeling grouchy,3
...,...,...
15995,i just had a very brief time in the beanbag an...,0
15996,i am now turning and i feel pathetic that i am...,0
15997,i feel strong and good overall,1
15998,i feel like this was such a rude comment and i...,3


# **LIWC**

In [22]:
import re
import torch
from collections import Counter, defaultdict

from datasets import load_dataset
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report

########################################
# 1) Basic tokenizer
########################################
def basic_tokenizer(text):
    return re.findall(r'\b\w+\b', text.lower())

########################################
# 2) Load dataset
########################################
emotions = load_dataset("emotion")  # 'train', 'validation', 'test'
train_ds = emotions["train"]
test_ds = emotions["test"]

########################################
# 3) Label Encode target
########################################
label_encoder = LabelEncoder()
train_labels = label_encoder.fit_transform(train_ds["label"])
test_labels  = label_encoder.transform(test_ds["label"])
num_classes  = len(label_encoder.classes_)

# Convert integer labels -> string labels for easy comparison
test_labels_str = [label_encoder.classes_[idx] for idx in test_labels]

########################################
# 4) Define keyword dictionaries
########################################

sadness_keywords = {
    "sad", "depressed", "unhappy", "miserable", "downcast", "blue",
    "heartbroken", "sorrowful", "gloomy", "despondent", "melancholy",
    "tearful", "weepy", "doleful", "woeful", "grieving", "mournful",
    "somber", "pensive", "lamenting"
}

anger_keywords = {
    "angry", "mad", "furious", "irritated", "irate", "enraged",
    "annoyed", "livid", "fuming", "vexed", "cross", "incensed",
    "resentful", "offended", "indignant", "irritable", "infuriated",
    "outraged", "wrathful", "aggravated"
}

love_keywords = {
    "love", "adore", "cherish", "fond", "devoted", "affectionate",
    "attached", "infatuated", "enamored", "passionate", "romantic",
    "amorous", "dear", "beloved", "sweetheart", "darling", "treasure",
    "precious", "admire", "yearn"
}

fear_keywords = {
    "afraid", "scared", "fearful", "terrified", "petrified", "nervous",
    "anxious", "frightened", "alarmed", "panic", "apprehensive",
    "trepidation", "intimidated", "spooked", "horrified", "dread",
    "fearsome", "uneasy", "scary", "wary"
}

surprise_keywords = {
    "surprised", "astonished", "amazed", "startled", "astounded",
    "stunned", "shocked", "flabbergasted", "speechless", "wow",
    "unbelievable", "marvel", "unexpected", "whoa", "remarkable",
    "dumbfounded", "perplexed", "aghast", "floored", "taken"
}

joy_keywords = {
    "happy", "joyful", "glad", "delighted", "elated", "pleased",
    "cheerful", "thrilled", "ecstatic", "blissful", "content",
    "jubilant", "sunny", "merry", "gleeful", "lighthearted",
    "exultant", "rejoicing", "rejoice", "radiant"
}

# For clarity, store them in a dictionary keyed by the emotion label
keyword_dicts = {
    "0": sadness_keywords,
    "3": anger_keywords,
    "2": love_keywords,
    "4": fear_keywords,
    "5": surprise_keywords,
    "1": joy_keywords
}

########################################
# 5) Rule-based classification function
########################################
def rule_based_classify(text):
    """Classify a piece of text by counting its tokens in each keyword dictionary."""
    tokens = basic_tokenizer(text)
    # Count matches for each emotion
    emotion_counts = {}
    for emotion, word_set in keyword_dicts.items():
        count = sum(token in word_set for token in tokens)
        emotion_counts[emotion] = count

    # Pick the emotion with the highest count
    predicted_emotion = max(emotion_counts, key=emotion_counts.get)
    return predicted_emotion

########################################
# 6) Apply rule-based classifier to the test set
########################################
all_preds = []
all_texts = test_ds["text"]  # The raw test text

for text in all_texts:
    pred = rule_based_classify(text)
    all_preds.append(pred)

all_preds_int = [label_encoder.transform([pred])[0] for pred in all_preds]

########################################
# 7) Evaluate
########################################
print("Classification Report (Rule-based):")
# Then you can do
print(classification_report(
    test_labels,        # numeric
    all_preds_int,      # numeric
    labels=[0,1,2,3,4,5]
))


Classification Report (Rule-based):
              precision    recall  f1-score   support

           0       0.33      0.94      0.48       581
           1       0.85      0.09      0.16       695
           2       0.37      0.21      0.27       159
           3       0.80      0.19      0.31       275
           4       0.79      0.25      0.39       224
           5       0.58      0.23      0.33        66

    accuracy                           0.38      2000
   macro avg       0.62      0.32      0.32      2000
weighted avg       0.64      0.38      0.31      2000



# **Naive Bayes**

In [9]:
from datasets import load_dataset
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.metrics import classification_report, accuracy_score

# Load the dataset
emotions = load_dataset("emotion")
train_ds = emotions["train"]
test_ds = emotions["test"]

# Extract features and labels
X_train = train_ds["text"]
y_train = train_ds["label"]
X_test = test_ds["text"]
y_test = test_ds["label"]

# Create a pipeline with CountVectorizer and MultinomialNB
model = make_pipeline(CountVectorizer(), MultinomialNB())

# Train the model
model.fit(X_train, y_train)

# Predict on test data
y_pred = model.predict(X_test)

# Print evaluation metrics
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=emotions["train"].features["label"].names))

Accuracy: 0.7655

Classification Report:
              precision    recall  f1-score   support

     sadness       0.74      0.94      0.83       581
         joy       0.74      0.97      0.84       695
        love       0.95      0.23      0.37       159
       anger       0.92      0.57      0.70       275
        fear       0.82      0.53      0.64       224
    surprise       0.00      0.00      0.00        66

    accuracy                           0.77      2000
   macro avg       0.69      0.54      0.56      2000
weighted avg       0.77      0.77      0.73      2000



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


# **LSTM**

In [9]:
!pip install torch torchvision torchaudio datasets scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 111.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 91.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 32.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 53.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitl

In [10]:
!pip install torchtext

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 20.7 MB/s eta 0:00:00


In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from datasets import load_dataset
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from torch.nn.utils.rnn import pad_sequence
from collections import Counter
from collections import defaultdict
import re

class SimpleVocab:
    def __init__(self, counter, specials=["<unk>", "<pad>"]):
        self.itos = specials + sorted(counter)
        self.stoi = defaultdict(lambda: self.itos.index("<unk>"), {tok: idx for idx, tok in enumerate(self.itos)})

    def __getitem__(self, token):
        return self.stoi[token]

    def __len__(self):
        return len(self.itos)

    def set_default_index(self, idx):
        self.stoi.default_factory = lambda: idx


def basic_tokenizer(text):
    return re.findall(r'\b\w+\b', text.lower())

tokenizer = basic_tokenizer

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load dataset
emotions = load_dataset("emotion")
train_ds = emotions["train"]
test_ds = emotions["test"]

# Preprocess labels
label_encoder = LabelEncoder()
train_labels = label_encoder.fit_transform(train_ds["label"])
test_labels = label_encoder.transform(test_ds["label"])
num_classes = len(label_encoder.classes_)

counter = Counter()
# Tokenize
for line in train_ds["text"]:
    counter.update(tokenizer(line))

vocab = SimpleVocab(counter)
vocab.set_default_index(vocab["<unk>"])

# Encode text
def encode_text(text):
    return torch.tensor([vocab[token] for token in tokenizer(text)], dtype=torch.long)

# Custom Dataset
class EmotionDataset(Dataset):
    def __init__(self, texts, labels):
        self.texts = [encode_text(t) for t in texts]
        self.labels = torch.tensor(labels, dtype=torch.long)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.texts[idx], self.labels[idx]

# Collate for DataLoader
def collate_batch(batch):
    texts, labels = zip(*batch)
    texts_padded = pad_sequence(texts, batch_first=True, padding_value=vocab['<pad>'])
    return texts_padded.to(device), torch.tensor(labels).to(device)

# DataLoader
train_data = EmotionDataset(train_ds["text"], train_labels)
test_data = EmotionDataset(test_ds["text"], test_labels)

train_loader = DataLoader(train_data, batch_size=64, shuffle=True, collate_fn=collate_batch)
test_loader = DataLoader(test_data, batch_size=64, shuffle=False, collate_fn=collate_batch)

# LSTM Model
class LSTMClassifier(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, output_dim):
        super(LSTMClassifier, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=vocab['<pad>'])
        self.lstm = nn.LSTM(embed_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        embedded = self.embedding(x)
        _, (hidden, _) = self.lstm(embedded)
        return self.fc(hidden[-1])

model = LSTMClassifier(len(vocab), embed_dim=100, hidden_dim=128, output_dim=num_classes).to(device)

# Training
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
epochs = 20

for epoch in range(epochs):
    model.train()
    total_loss = 0
    for texts, labels in train_loader:
        optimizer.zero_grad()
        output = model(texts)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {total_loss:.4f}")

# Evaluation
model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for texts, labels in test_loader:
        outputs = model(texts)
        preds = torch.argmax(outputs, dim=1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

target_names = [str(label) for label in label_encoder.classes_]
print(classification_report(all_labels, all_preds, target_names=target_names))

Epoch 1, Loss: 397.6643
Epoch 2, Loss: 393.9172
Epoch 3, Loss: 393.0453
Epoch 4, Loss: 392.2024
Epoch 5, Loss: 387.1522
Epoch 6, Loss: 332.3514
Epoch 7, Loss: 277.3054
Epoch 8, Loss: 250.7618
Epoch 9, Loss: 229.4399
Epoch 10, Loss: 200.2146
Epoch 11, Loss: 151.0209
Epoch 12, Loss: 121.2375
Epoch 13, Loss: 106.5093
Epoch 14, Loss: 96.9368
Epoch 15, Loss: 89.6840
Epoch 16, Loss: 86.3742
Epoch 17, Loss: 76.2703
Epoch 18, Loss: 56.7938
Epoch 19, Loss: 36.7022
Epoch 20, Loss: 25.9231
              precision    recall  f1-score   support

           0       0.93      0.92      0.92       581
           1       0.88      0.95      0.91       695
           2       0.80      0.62      0.70       159
           3       0.82      0.85      0.84       275
           4       0.87      0.83      0.85       224
           5       0.75      0.62      0.68        66

    accuracy                           0.88      2000
   macro avg       0.84      0.80      0.82      2000
weighted avg       0.87     

# **RNN**

In [16]:
import re
from collections import Counter, defaultdict

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence

from datasets import load_dataset
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report

########################################
# 1) Basic tokenizer
########################################
def basic_tokenizer(text):
    return re.findall(r'\b\w+\b', text.lower())

########################################
# 2) Simple vocabulary
########################################
class SimpleVocab:
    def __init__(self, counter, specials=["<unk>", "<pad>"]):
        # itos: index-to-string
        self.itos = specials + sorted(counter)
        # stoi: string-to-index (default to <unk> if unseen)
        unk_index = self.itos.index("<unk>")
        self.stoi = defaultdict(lambda: unk_index,
                                {tok: idx for idx, tok in enumerate(self.itos)})

    def __getitem__(self, token):
        return self.stoi[token]

    def __len__(self):
        return len(self.itos)

    def set_default_index(self, idx):
        """Change what the default index should be for unknown tokens."""
        self.stoi.default_factory = lambda: idx

########################################
# 3) Load dataset and device
########################################
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
emotions = load_dataset("emotion")

train_ds = emotions["train"]
test_ds  = emotions["test"]

########################################
# 4) Label encode target
########################################
label_encoder = LabelEncoder()
train_labels = label_encoder.fit_transform(train_ds["label"])
test_labels  = label_encoder.transform(test_ds["label"])
num_classes  = len(label_encoder.classes_)

########################################
# 5) Build vocabulary from training data
########################################
counter = Counter()
for line in train_ds["text"]:
    counter.update(basic_tokenizer(line))

vocab = SimpleVocab(counter)
vocab.set_default_index(vocab["<unk>"])

########################################
# 6) Encode text function
########################################
def encode_text(text):
    tokens = basic_tokenizer(text)
    return torch.tensor([vocab[tok] for tok in tokens], dtype=torch.long)

########################################
# 7) Custom Dataset
########################################
class EmotionDataset(Dataset):
    def __init__(self, texts, labels):
        self.texts  = [encode_text(t) for t in texts]
        self.labels = torch.tensor(labels, dtype=torch.long)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.texts[idx], self.labels[idx]

########################################
# 8) Collate function for DataLoader
########################################
def collate_batch(batch):
    texts, labels = zip(*batch)
    # Pad sequences to the same length
    texts_padded = pad_sequence(texts, batch_first=True, padding_value=vocab['<pad>'])
    return texts_padded.to(device), torch.tensor(labels).to(device)

########################################
# 9) Create train/test DataLoaders
########################################
train_data = EmotionDataset(train_ds["text"], train_labels)
test_data  = EmotionDataset(test_ds["text"], test_labels)

train_loader = DataLoader(train_data, batch_size=64, shuffle=True, collate_fn=collate_batch)
test_loader  = DataLoader(test_data, batch_size=64, shuffle=False, collate_fn=collate_batch)

########################################
# 10) RNN Classifier
########################################
class RNNClassifier(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, output_dim):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=vocab["<pad>"])
        self.rnn = nn.RNN(embed_dim, hidden_dim, batch_first=True)
        self.fc  = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        # x shape: [batch_size, seq_length]
        embedded = self.embedding(x)  # [batch_size, seq_length, embed_dim]
        # out: [batch_size, seq_length, hidden_dim]
        # hidden: [1, batch_size, hidden_dim] (for 1-layer RNN)
        out, hidden = self.rnn(embedded)
        last_output = out[:, -1, :]  # final time-step
        return self.fc(last_output)


########################################
# 11) Initialize model, loss, optimizer
########################################
model = RNNClassifier(
    vocab_size=len(vocab),
    embed_dim=100,
    hidden_dim=128,
    output_dim=num_classes
).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

########################################
# 12) Training loop
########################################
epochs = 20
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for batch_texts, batch_labels in train_loader:
        optimizer.zero_grad()
        outputs = model(batch_texts)
        loss = criterion(outputs, batch_labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    print(f"Epoch [{epoch+1}/{epochs}] - Loss: {avg_loss:.4f}")

########################################
# 13) Evaluation
########################################
model.eval()
all_preds  = []
all_labels = []

with torch.no_grad():
    for batch_texts, batch_labels in test_loader:
        outputs = model(batch_texts)
        preds   = torch.argmax(outputs, dim=1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(batch_labels.cpu().numpy())

print("\nClassification Report:")
print(
    classification_report(
        all_labels,
        all_preds,
    )
)

target_names = [str(label) for label in label_encoder.classes_]
print(classification_report(all_labels, all_preds, target_names=target_names))

Epoch [1/20] - Loss: 1.5872
Epoch [2/20] - Loss: 1.5765
Epoch [3/20] - Loss: 1.5762
Epoch [4/20] - Loss: 1.5727
Epoch [5/20] - Loss: 1.5727
Epoch [6/20] - Loss: 1.5691
Epoch [7/20] - Loss: 1.5677
Epoch [8/20] - Loss: 1.5685
Epoch [9/20] - Loss: 1.5662
Epoch [10/20] - Loss: 1.5661
Epoch [11/20] - Loss: 1.5664
Epoch [12/20] - Loss: 1.5650
Epoch [13/20] - Loss: 1.5653
Epoch [14/20] - Loss: 1.5642
Epoch [15/20] - Loss: 1.5632
Epoch [16/20] - Loss: 1.5635
Epoch [17/20] - Loss: 1.5616
Epoch [18/20] - Loss: 1.5630
Epoch [19/20] - Loss: 1.5606
Epoch [20/20] - Loss: 1.5608

Classification Report:
              precision    recall  f1-score   support

           0       0.40      0.03      0.05       581
           1       0.35      0.97      0.51       695
           2       0.00      0.00      0.00       159
           3       0.12      0.00      0.01       275
           4       0.00      0.00      0.00       224
           5       0.00      0.00      0.00        66

    accuracy             

# **CNN**

In [12]:
import re
from collections import Counter, defaultdict

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence

from datasets import load_dataset
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report

########################################
# 1) Basic tokenizer
########################################
def basic_tokenizer(text):
    return re.findall(r'\b\w+\b', text.lower())

########################################
# 2) Simple vocabulary
########################################
class SimpleVocab:
    def __init__(self, counter, specials=["<unk>", "<pad>"]):
        self.itos = specials + sorted(counter)
        unk_index = self.itos.index("<unk>")
        self.stoi = defaultdict(lambda: unk_index,
                                {tok: idx for idx, tok in enumerate(self.itos)})

    def __getitem__(self, token):
        return self.stoi[token]

    def __len__(self):
        return len(self.itos)

    def set_default_index(self, idx):
        self.stoi.default_factory = lambda: idx

########################################
# 3) Load dataset and device
########################################
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
emotions = load_dataset("emotion")  # 'train', 'validation', 'test'

train_ds = emotions["train"]
test_ds  = emotions["test"]

########################################
# 4) Label encode target
########################################
label_encoder = LabelEncoder()
train_labels = label_encoder.fit_transform(train_ds["label"])
test_labels  = label_encoder.transform(test_ds["label"])
num_classes  = len(label_encoder.classes_)

########################################
# 5) Build vocabulary from training data
########################################
counter = Counter()
for line in train_ds["text"]:
    counter.update(basic_tokenizer(line))

vocab = SimpleVocab(counter)
vocab.set_default_index(vocab["<unk>"])

########################################
# 6) Encode text function
########################################
def encode_text(text):
    tokens = basic_tokenizer(text)
    return torch.tensor([vocab[tok] for tok in tokens], dtype=torch.long)

########################################
# 7) Custom Dataset
########################################
class EmotionDataset(Dataset):
    def __init__(self, texts, labels):
        self.texts  = [encode_text(t) for t in texts]
        self.labels = torch.tensor(labels, dtype=torch.long)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.texts[idx], self.labels[idx]

########################################
# 8) Collate function for DataLoader
########################################
def collate_batch(batch):
    texts, labels = zip(*batch)
    # Pad to same length along seq dimension
    texts_padded = pad_sequence(texts, batch_first=True, padding_value=vocab['<pad>'])
    return texts_padded.to(device), torch.tensor(labels).to(device)

########################################
# 9) Create train/test DataLoaders
########################################
train_data = EmotionDataset(train_ds["text"], train_labels)
test_data  = EmotionDataset(test_ds["text"], test_labels)

train_loader = DataLoader(train_data, batch_size=64, shuffle=True, collate_fn=collate_batch)
test_loader  = DataLoader(test_data, batch_size=64, shuffle=False, collate_fn=collate_batch)

########################################
# 10) CNN Classifier
########################################
class CNNClassifier(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_filters, output_dim, kernel_size=3):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=vocab["<pad>"])

        self.conv = nn.Conv1d(
            in_channels=embed_dim,
            out_channels=num_filters,
            kernel_size=kernel_size
        )
        self.fc = nn.Linear(num_filters, output_dim)

    def forward(self, x):
        # x shape: [batch_size, seq_len]
        embedded = self.embedding(x)          # [batch_size, seq_len, embed_dim]
        embedded = embedded.permute(0, 2, 1)  # => [batch_size, embed_dim, seq_len]

        # Convolution + ReLU
        conved = self.conv(embedded)          # => [batch_size, num_filters, seq_len - kernel_size + 1]
        activated = F.relu(conved)

        # Global max pooling across the time dimension
        pooled = F.max_pool1d(activated, kernel_size=activated.shape[2])  # => [batch_size, num_filters, 1]
        pooled = pooled.squeeze(2)  # => [batch_size, num_filters]

        # Final linear for classification
        output = self.fc(pooled)  # => [batch_size, output_dim]
        return output

########################################
# 11) Initialize model, loss, optimizer
########################################
model = CNNClassifier(
    vocab_size=len(vocab),
    embed_dim=100,      # size of the word embeddings
    num_filters=100,    # number of output channels (filters) in the CNN
    output_dim=num_classes,
    kernel_size=3
).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

########################################
# 12) Training loop
########################################
epochs = 20
for epoch in range(epochs):
    model.train()
    total_loss = 0

    for batch_texts, batch_labels in train_loader:
        optimizer.zero_grad()
        outputs = model(batch_texts)
        loss = criterion(outputs, batch_labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    print(f"Epoch [{epoch+1}/{epochs}] - Loss: {avg_loss:.4f}")

########################################
# 13) Evaluation
########################################
model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for batch_texts, batch_labels in test_loader:
        outputs = model(batch_texts)
        preds = torch.argmax(outputs, dim=1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(batch_labels.cpu().numpy())

print("\nClassification Report:")
print(
    classification_report(
        all_labels,
        all_preds,
    )
)

target_names = [str(label) for label in label_encoder.classes_]
print(classification_report(all_labels, all_preds, target_names=target_names))

Epoch [1/20] - Loss: 1.4741
Epoch [2/20] - Loss: 0.9415
Epoch [3/20] - Loss: 0.4409
Epoch [4/20] - Loss: 0.2191
Epoch [5/20] - Loss: 0.1259
Epoch [6/20] - Loss: 0.0765
Epoch [7/20] - Loss: 0.0494
Epoch [8/20] - Loss: 0.0350
Epoch [9/20] - Loss: 0.0272
Epoch [10/20] - Loss: 0.0226
Epoch [11/20] - Loss: 0.0214
Epoch [12/20] - Loss: 0.0209
Epoch [13/20] - Loss: 0.0196
Epoch [14/20] - Loss: 0.0188
Epoch [15/20] - Loss: 0.0198
Epoch [16/20] - Loss: 0.0196
Epoch [17/20] - Loss: 0.0179
Epoch [18/20] - Loss: 0.0226
Epoch [19/20] - Loss: 0.0205
Epoch [20/20] - Loss: 0.0212

Classification Report:
              precision    recall  f1-score   support

           0       0.94      0.91      0.93       581
           1       0.88      0.93      0.91       695
           2       0.76      0.74      0.75       159
           3       0.88      0.89      0.89       275
           4       0.90      0.82      0.86       224
           5       0.64      0.67      0.65        66

    accuracy             